In [25]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Set display options to avoid scientific notation, format as float with 2 decimal places
pd.set_option('display.float_format', '{:.2f}'.format)

# Load the dataset
data = pd.read_excel(r'C:\Users\asus\Desktop\Assignment\Second Assignment\Mod Global Economy Indicator 1.xlsx')

# Relevant columns
relevant_columns = [
    'Country', 'Year',
    'Agriculture, hunting, forestry, fishing (ISIC A-B)', 'Construction (ISIC F)',
    'Exports of goods and services', 'Imports of goods and services',
    'Manufacturing (ISIC D)', 'Mining, Manufacturing, Utilities (ISIC C-E)',
    'Other Activities (ISIC J-P)', 'Transport, storage and communication (ISIC I)',
    'Wholesale, retail trade, restaurants and hotels (ISIC G-H)',
    'Gross National Income(GNI) in USD', 'Gross Domestic Product (GDP)',
    'Trade Balance', 'Sectoral Output'
]

# Keep only relevant columns
data = data[relevant_columns]
# Descriptive statistics for specific columns before preprocessing
print("Descriptive Statistics Before Preprocessing (Specific Columns):")
print(data[['Gross Domestic Product (GDP)', 'Gross National Income(GNI) in USD', 'Trade Balance', 'Sectoral Output']].describe())

# Handling missing values for numerical and categorical data
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Remove outliers using IQR
Q1 = data[numeric_cols].quantile(0.25)
Q3 = data[numeric_cols].quantile(0.75)
IQR = Q3 - Q1
data = data[~((data[numeric_cols] < (Q1 - 1.5 * IQR)) |(data[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

numeric_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# We will use a pipeline to apply transformations step by step
pipeline = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', numeric_imputer),
            ('scaler', RobustScaler()),
            ('power_transformer', PowerTransformer(method='yeo-johnson'))]), numeric_cols),
        ('cat', categorical_imputer, categorical_cols)
    ]
)

data_transformed = pipeline.fit_transform(data)
data_transformed = pd.DataFrame(data_transformed, columns=numeric_cols.tolist() + categorical_cols.tolist())

# Apply PCA for dimensionality reduction to the numeric columns
pca = PCA(n_components=0.95)  # Retain 95% of variance
data_pca = pca.fit_transform(data_transformed[numeric_cols])
data_pca = pd.DataFrame(data_pca, columns=[f'PCA_{i}' for i in range(data_pca.shape[1])])

# Converting the transformed data back to a DataFrame
data_transformed = pd.DataFrame(data_transformed, columns=numeric_cols.tolist() + categorical_cols.tolist())

# Converting the transformed data back to a DataFrame
data_transformed = pd.DataFrame(data_transformed, columns=numeric_cols.tolist() + categorical_cols.tolist())

# Descriptive statistics for specific columns after preprocessing
print("\nDescriptive Statistics After Preprocessing (Specific Columns):")
# After applying transformations, ensure the columns are of numeric type
for col in ['Gross National Income(GNI) in USD', 'Gross Domestic Product (GDP)','Trade Balance', 'Sectoral Output']:
    data_transformed[col] = pd.to_numeric(data_transformed[col], errors='coerce')

# Now, when you use describe(), it should include all metrics for numerical columns
descriptive_stats = data_transformed.describe()

print(descriptive_stats)

# Convert all columns to numeric if possible
data_transformed = data_transformed.apply(pd.to_numeric, errors='ignore')

# Export the pre-processed dataset without index
data_transformed.to_csv(r'C:\Users\asus\Desktop\Assignment\Second Assignment\Preprocessed1.csv', index=False, quotechar='"')

print("\nOutlier removal, preprocessing, and PCA transformation complete.")


Descriptive Statistics Before Preprocessing (Specific Columns):
       Gross Domestic Product (GDP)  Gross National Income(GNI) in USD  \
count                      10512.00                           10512.00   
mean                182876492569.56                    182526026614.44   
std                 984599654555.47                    994214438370.94   
min                      2585174.00                         3564833.00   
25%                   1439216839.50                      1410771595.25   
50%                   8070571563.00                      7887233337.50   
75%                  51725961801.75                     50057889497.50   
max               23300000000000.00                  23600000000000.00   

         Trade Balance   Sectoral Output  
count         10512.00          10512.00  
mean      886852797.28   205276247405.42  
std     33725034708.46  1135340497612.55  
min   -860000000000.00        2458531.00  
25%      -797478570.25     1511274781.25  
50%       -